# LangChain

In [1]:
import os
import json
import pymongo
from typing import List
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import AzureCosmosDBVectorSearch
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.schema.document import Document
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.agents import Tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain_core.messages import SystemMessage

In [2]:
# Load settings for the notebook
load_dotenv()
CONNECTION_STRING = os.environ.get("DB_CONNECTION_STRING")
EMBEDDINGS_DEPLOYMENT_NAME = "text-embedding-3-small"
COMPLETIONS_DEPLOYMENT_NAME = "gpt-35-turbo-16k"
AOAI_ENDPOINT = os.environ.get("AOAI_ENDPOINT")
AOAI_KEY = os.environ.get("AOAI_KEY")
AOAI_API_VERSION = "2023-09-01-preview"

In [3]:
# Establish Azure OpenAI connectivity
llm = AzureChatOpenAI(            
        temperature = 0,
        openai_api_version = AOAI_API_VERSION,
        azure_endpoint = AOAI_ENDPOINT,
        openai_api_key = AOAI_KEY,         
        azure_deployment = COMPLETIONS_DEPLOYMENT_NAME
)
embedding_model = AzureOpenAIEmbeddings(
    openai_api_version = AOAI_API_VERSION,
    azure_endpoint = AOAI_ENDPOINT,
    openai_api_key = AOAI_KEY,   
    azure_deployment = EMBEDDINGS_DEPLOYMENT_NAME,
    chunk_size=10
)

/home/norbi/repos/ai-learning-hackathon-phase1/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/home/norbi/repos/ai-learning-hackathon-phase1/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


## Vector search with LangChain

In the previous lab, the `pymongo` library was used to perform a vector search through a db command to find product documents that were most similar to the user's input. In this lab, you will use the `langchain` library to perform the same search. LangChain has a vector store class named **AzureCosmosDBVectorSearch**, a community contribution, that supports vector search in vCore-based Azure Cosmos DB for MongoDB.

When establishing the connection to the vector store (vCore-based Azure Cosmos DB for MongoDB), recall that in previous labs the products collection was populated and a contentVector field added that contains the vectorized embeddings of the document itself. Finally, a vector index was also created on the contentVector field to enable vector search. The vector index in each collection is named `VectorSearchIndex`.

The return value of a vector search in LangChain is a list of `Document` objects. The LangChain `Document` class contains two properties: `page_content`, that represents the textual content that is typically used to augment the prompt, and `metadata` that contains all other attributes of the document. In the cell below, we'll use the `_id` field as the page_content, and the rest of the fields are returned as metadata.

The next two cells initiate a connection to the vector store and performs a vector search. Notice how much more concise the code is compared to the previous lab with the addition of LangChain.

In [4]:
# Reference the existing vector store
vector_store = AzureCosmosDBVectorSearch.from_connection_string(
    connection_string = CONNECTION_STRING,
    namespace = "cosmic_works.products",
    embedding = embedding_model,
    index_name = "VectorSearchIndex",    
    embedding_key = "contentVector",
    text_key = "_id"
)

/home/norbi/repos/ai-learning-hackathon-phase1/venv/lib/python3.12/site-packages/langchain_community/vectorstores/azure_cosmos_db.py:145: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client: MongoClient = MongoClient(connection_string, appname=appname)


In [5]:
query = "What yellow products are there?"
vector_store.similarity_search(query, k=3)

[Document(page_content='5B5E90B8-FEA2-4D6C-B728-EC586656FA6D', metadata={'categoryId': '75BF1ACB-168D-469C-9AA3-1FD26BB4EA4C', 'categoryName': 'Bikes, Touring Bikes', 'sku': 'BK-T79Y-60', 'name': 'Touring-1000 Yellow, 60', 'description': 'The product called "Touring-1000 Yellow, 60"', 'price': 2384.07, 'tags': [{'_id': '4F67013C-3B5E-4A3D-B4B0-8C597A491EB6', 'name': 'Tag-82'}, {'_id': '6C6D061E-F701-41DC-AEA2-7A5C28061840', 'name': 'Tag-98'}], 'contentVector': [-0.0038613835, -0.022302395, -0.04035269, -0.018810606, -0.025625225, -0.006332387, -0.0036431467, 0.013382845, 0.0077086873, -0.029792845, 0.009123706, -0.03812808, -0.02673753, -0.033340953, 0.026807928, 0.050405663, -0.0073637324, -0.0114046335, -0.019739872, 0.06409122, 0.04578749, -0.032439847, 0.048659768, 0.028384864, 0.011291995, -0.028553823, -0.018022137, -0.0022193275, 0.051475726, -0.05879722, -0.022710707, -0.020992974, 0.035818998, -0.006508385, -0.04085956, -0.013565882, 0.027680876, 0.0056424774, 0.03838152, 0.04

## RAG with LangChain

In this section, we'll implement the RAG pattern using LangChain. In LangChain, a **retriever** is used to augment the prompt with contextual data. In this case, the already established vector store will be used as the retriever. By default, the prompt is augmented with the `page_content` field of the retrieved document that customarily contains the text content of the embedded vector. In our case, the document itself serves as the textual content, so we'll have to do some pre-processing to format the text of the product list that is expected in our system prompt (JSON string) - see the **format_documents** function below for this implementation.

We'll also define a reusable RAG [chain](https://python.langchain.com/docs/modules/chains/) to control the flow and behavior of the call into the LLM. This chain is defined using the LCEL syntax (LangChain Expression Language).

In [6]:
# A system prompt describes the responsibilities, instructions, and persona of the AI.
# Note the addition of the templated variable/placeholder for the list of products and the incoming question.
system_prompt = """
You are a helpful, fun and friendly sales assistant for Cosmic Works, a bicycle and bicycle accessories store. 
Your name is Cosmo.
You are designed to answer questions about the products that Cosmic Works sells.

Only answer questions related to the information provided in the list of products below that are represented
in JSON format.

If you are asked a question that is not in the list, respond with "I don't know."

Only answer questions related to Cosmic Works products, customers, and sales orders.

If a question is not related to Cosmic Works products, customers, or sales orders,
respond with "I only answer questions about Cosmic Works"

List of products:
{products}

Question:
{question}
"""

In [7]:
# remember that each Document contains a page_content property
# that is populated with the _id field of the document
# all other document fields are located in the metadata property
def format_docs(docs:List[Document]) -> str:
        """
        Prepares the product list for the system prompt.
        """
        str_docs = []
        for doc in docs:
                # Build the product document without the contentVector
                doc_dict = {"_id": doc.page_content}
                doc_dict.update(doc.metadata)
                if "contentVector" in doc_dict:  
                        del doc_dict["contentVector"]
                str_docs.append(json.dumps(doc_dict, default=str))                  
        # Return a single string containing each product JSON representation
        # separated by two newlines
        return "\n\n".join(str_docs)

In [8]:
# Create a retriever from the vector store
retriever = vector_store.as_retriever()

# Create the prompt template from the system_prompt text
llm_prompt = PromptTemplate.from_template(system_prompt)

rag_chain = (
    # populate the tokens/placeholders in the llm_prompt 
    # products takes the results of the vector store and formats the documents
    # question is a passthrough that takes the incoming question
    { "products": retriever | format_docs, "question": RunnablePassthrough()}
    | llm_prompt
    # pass the populated prompt to the language model
    | llm
    # return the string ouptut from the language model
    | StrOutputParser()
)

In [9]:
question = "What are the names and skus of yellow products? Output the answer as a bulleted list."
response = rag_chain.invoke(question)
print(response)

- Touring-3000 Yellow, 58 (SKU: BK-T18Y-58)
- Touring-1000 Yellow, 50 (SKU: BK-T79Y-50)
- Touring-1000 Yellow, 60 (SKU: BK-T79Y-60)
- Road-550-W Yellow, 40 (SKU: BK-R64Y-40)


## LangChain Agent

### Create retrievers

A separate retriever is required for each vector index. The following cell creates a VectorStoreRetriever for the products, customers, and sales collections and associated vector index.

In [10]:
def create_cosmic_works_vector_store_retriever(collection_name: str, top_k: int = 3):
    vector_store =  AzureCosmosDBVectorSearch.from_connection_string(
        connection_string = CONNECTION_STRING,
        namespace = f"cosmic_works.{collection_name}",
        embedding = embedding_model,
        index_name = "VectorSearchIndex",    
        embedding_key = "contentVector",
        text_key = "_id"
    )
    return vector_store.as_retriever(search_kwargs={"k": top_k})


products_retriever = create_cosmic_works_vector_store_retriever("products")
customers_retriever = create_cosmic_works_vector_store_retriever("customers")
sales_retriever = create_cosmic_works_vector_store_retriever("sales")

/home/norbi/repos/ai-learning-hackathon-phase1/venv/lib/python3.12/site-packages/langchain_community/vectorstores/azure_cosmos_db.py:145: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client: MongoClient = MongoClient(connection_string, appname=appname)


 ### Create Agent Tools
 
 LangChain does have a built-in [`create_retriever_tool`](https://python.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents#retriever-tool) that wraps a vector store retriever, however, because we are storing the embeddings in the `contentVector` field of the document, we must do some pre-processing of the retrieved documents to remove this field so that we don't needlessly expend the model's token quota. 
 
 Instead, we'll create a RAG chain as our tool implementation that does the pre-processing through the `format_docs` function we defined above to return each document in its JSON representation.

In [11]:
# Create tools that will use vector search in vCore-based Azure Cosmos DB for MongoDB collections

# create a chain on the retriever to format the documents as JSON
products_retriever_chain = products_retriever | format_docs
customers_retriever_chain = customers_retriever | format_docs
sales_retriever_chain = sales_retriever | format_docs

tools = [
    Tool(
        name = "vector_search_products", 
        func = products_retriever_chain.invoke,
        description = "Searches Cosmic Works product information for similar products based on the question. Returns the product information in JSON format."
    ),
    Tool(
        name = "vector_search_customers", 
        func = customers_retriever_chain.invoke,
        description = "Searches Cosmic Works customer information and retrieves similar customers based on the question. Returns the customer information in JSON format."
    ),
    Tool(
        name = "vector_search_sales", 
        func = sales_retriever_chain.invoke,
        description = "Searches Cosmic Works customer sales information and retrieves sales order details based on the question. Returns the sales order information in JSON format."
    )
]

### Tools part 2

Certain properties do not have semantic meaning (such as the GUID _id fields) and attempting to use vector search on these fields will not yield meaningful results. We need a tool to retrieve specific documents based on popular searches criteria.

The following tool definitions is not an exhaustive list of what may be needed but serves as an example to provide concrete lookups of a document in the Cosmic Works database.

In [12]:
db = pymongo.MongoClient(CONNECTION_STRING).cosmic_works

def get_product_by_id(product_id: str) -> str:
    """
    Retrieves a product by its ID.    
    """
    doc = db.products.find_one({"_id": product_id})    
    if "contentVector" in doc:
        del doc["contentVector"]
    return json.dumps(doc)

def get_product_by_sku(sku: str) -> str:
    """
    Retrieves a product by its sku.
    """
    doc = db.products.find_one({"sku": sku})
    if "contentVector" in doc:
        del doc["contentVector"]
    return json.dumps(doc, default=str)

def get_sales_by_id(sales_id: str) -> str:
    """
    Retrieves a sales order by its ID.
    """
    doc = db.sales.find_one({"_id": sales_id})
    if "contentVector" in doc:
        del doc["contentVector"]
    return json.dumps(doc, default=str)    

from langchain.tools import StructuredTool

tools.extend([
    StructuredTool.from_function(get_product_by_id),
    StructuredTool.from_function(get_product_by_sku),
    StructuredTool.from_function(get_sales_by_id)
])

/tmp/ipykernel_9634/47035323.py:1: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  db = pymongo.MongoClient(CONNECTION_STRING).cosmic_works


### Create the agent

The [`create_conversational_retrieval_agent`](https://python.langchain.com/docs/use_cases/question_answering/conversational_retrieval_agents#agent-constructor) is a built-in agent that includes conversational history as well uses the [OpenAIFunctionsAgent](https://python.langchain.com/docs/modules/agents/agent_types/openai_functions_agent#using-openaifunctionsagent) as its underlying implementation.

Remember that an agent leverages the LLM to assess the incoming request with the current context to decide what steps or actions need to be executed and in what order. LangChain agents can leverage tools. A tool can be an integration into an external system, custom code, or even another chain.

In [13]:
system_message = SystemMessage(
    content = """
        You are a helpful, fun and friendly sales assistant for Cosmic Works, a bicycle and bicycle accessories store.

        Your name is Cosmo.

        You are designed to answer questions about the products that Cosmic Works sells, the customers that buy them, and the sales orders that are placed by customers.

        If you don't know the answer to a question, respond with "I don't know."
        
        Only answer questions related to Cosmic Works products, customers, and sales orders.
        
        If a question is not related to Cosmic Works products, customers, or sales orders,
        respond with "I only answer questions about Cosmic Works"
    """    
)
agent_executor = create_conversational_retrieval_agent(llm, tools, system_message = system_message, verbose=True)

In [14]:
result = agent_executor({"input": "What products do you have that are yellow?"})
print("***********************************************************")
print(result['output'])

/home/norbi/repos/ai-learning-hackathon-phase1/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `vector_search_products` with `yellow`


{"_id": "26E8185C-782A-4B48-87FA-1E715E3825FB", "categoryId": "AE48F0AA-4F65-4734-A4CF-D48B8F82267F", "categoryName": "Bikes, Road Bikes", "sku": "BK-R64Y-48", "name": "Road-550-W Yellow, 48", "description": "The product called \"Road-550-W Yellow, 48\"", "price": 1120.49, "tags": [{"_id": "46AC3482-E9A6-474D-A435-D3399F21991F", "name": "Tag-90"}, {"_id": "9C89E562-1247-435D-B786-4E54024E681C", "name": "Tag-128"}]}

{"_id": "91AA100C-D092-4190-92A7-7C02410F04EA", "categoryId": "F3FBB167-11D8-41E4-84B4-5AAA92B1E737", "categoryName": "Components, Touring Frames", "sku": "FR-T67Y-62", "name": "LL Touring Frame - Yellow, 62", "description": "The product called \"LL Touring Frame - Yellow, 62\"", "price": 333.42, "tags": [{"_id": "52FCE975-91EE-4789-9E36-94EC766F02A0", "name": "Tag-35"}]}

{"_id": "FFCA3096-199F-41C3-99D1-35BC88D8AC6F", "categoryId": "3E4CEACD-D007-46EB-82D7-31F6141752B2", "categoryN

In [15]:
result = agent_executor({"input": "What products were purchased for sales order '06FE91D2-B350-471A-AD29-906BF4EB97C4' ?"})
print("***********************************************************")
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_sales_by_id` with `{'sales_id': '06FE91D2-B350-471A-AD29-906BF4EB97C4'}`


{"_id": "06FE91D2-B350-471A-AD29-906BF4EB97C4", "customerId": "D133397E-83ED-4FF3-BD44-62D7481237B2", "orderDate": "2011-07-01 00:00:00", "shipDate": "2011-07-08 00:00:00", "details": [{"sku": "LJ-0192-L", "name": "Long-Sleeve Logo Jersey, L", "price": 28.8404, "quantity": 1}, {"sku": "BK-R68R-52", "name": "Road-450 Red, 52", "price": 874.794, "quantity": 1}, {"sku": "CA-1098", "name": "AWC Logo Cap", "price": 5.1865, "quantity": 1}]}
Invoking: `get_product_by_sku` with `{'sku': 'LJ-0192-L'}`


{"_id": "6401B68F-924A-4B2E-AC9E-5660AEA0E848", "categoryId": "C3C57C35-1D80-4EC5-AB12-46C57A017AFB", "categoryName": "Clothing, Jerseys", "sku": "LJ-0192-L", "name": "Long-Sleeve Logo Jersey, L", "description": "The product called \"Long-Sleeve Logo Jersey, L\"", "price": 49.99, "tags": [{"_id": "B48D6572-67EB-4630-A1DB-AFD4AD7041C9", "name": "Tag-100"}, {"_id": "E1

In [16]:
result = agent_executor({"input": "What was the sales order total for sales order '93436616-4C8A-407D-9FDA-908707EFA2C5' ?"})
print("***********************************************************")
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_sales_by_id` with `{'sales_id': '93436616-4C8A-407D-9FDA-908707EFA2C5'}`


{"_id": "93436616-4C8A-407D-9FDA-908707EFA2C5", "customerId": "6DF10977-EA10-4607-AF0F-74EDABA54943", "orderDate": "2013-09-30 00:00:00", "shipDate": "2013-10-07 00:00:00", "details": [{"sku": "BK-R89B-52", "name": "Road-250 Black, 52", "price": 1466.01, "quantity": 1}, {"sku": "HB-R956", "name": "HL Road Handlebars", "price": 72.162, "quantity": 1}, {"sku": "FR-R92B-44", "name": "HL Road Frame - Black, 44", "price": 858.9, "quantity": 3}, {"sku": "FR-R92R-44", "name": "HL Road Frame - Red, 44", "price": 858.9, "quantity": 1}]}The sales order total for sales order '93436616-4C8A-407D-9FDA-908707EFA2C5' is $4,114.12.

Let me know if there's anything else I can help with!

> Finished chain.
***********************************************************
The sales order total for sales order '93436616-4C8A-407D-9FDA-908707EFA2C5' is $4,114.12.

Let me know if ther

In [17]:
result = agent_executor({"input": "What was the price of the product with sku `FR-R92B-58` ?"})
print("***********************************************************")
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_product_by_sku` with `{'sku': 'FR-R92B-58'}`


{"_id": "2CE4EFA7-5DC6-4D3E-ACB2-B7DDE4518408", "categoryId": "3E4CEACD-D007-46EB-82D7-31F6141752B2", "categoryName": "Components, Road Frames", "sku": "FR-R92B-58", "name": "HL Road Frame - Black, 58", "description": "The product called \"HL Road Frame - Black, 58\"", "price": 1431.5, "tags": [{"_id": "2CE9DADE-DCAC-436C-9D69-B7C886A01B77", "name": "Tag-101"}, {"_id": "AEFA79EF-CBF1-4824-AAF7-1D20EA85119B", "name": "Tag-17"}, {"_id": "D197C394-FB88-4EFF-B0FB-8BED1A86F294", "name": "Tag-182"}, {"_id": "ECBBCC15-3016-4075-B084-4B49DA754814", "name": "Tag-138"}]}The price of the product with SKU 'FR-R92B-58' is $1431.50.

Let me know if there's anything else I can assist you with!

> Finished chain.
***********************************************************
The price of the product with SKU 'FR-R92B-58' is $1431.50.

Let me know if there's anything else I can assist you with!
